# Real-world Temporal Network Simulations
Here we will illustrate how the DelayBufferNetwork framework can be used to build temporal networks on real-world data. We use the Thiers13 data set here, from sociopatterns.org (http://www.sociopatterns.org/datasets/high-school-contact-and-friendship-networks/)

Keep in mind that this uses an older version of DelayBufferNetwork. Using the updated framework is recommended but it will not work in the same way.

Note that this network is more connected than the network build on the Workplace15 data, meaning that this notebook takes much longer to execute. 

First, load in the data (which was preprocessed from the original contact representation format (t: {agent_1, agent_2, ...}) to event format (i, j, t, event_id))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from DelayBufferNetwork import DelayBufferNetwork

In [ ]:
# Download the Thiers13 dataset from sociopatterns.org
# extract data
data = pd.read_csv("real_world_data/HighSchool2013_proximity_net.csv", delim_whitespace=True, header=None)
data.head()

In [ ]:
# Save to .npy
np.save("Thiers13/Thiers13.npy", data.values)
event_data = np.load("Thiers13/Thiers13.npy", allow_pickle=True)
event_data

In [ ]:
df = pd.DataFrame(event_data, columns=["t", "i", "j", "weight", "event_id"])
df['event_id'] = df.index.astype(int)
df.head(5)


In [ ]:
# Create a dictionary to map the old agent IDs to the new agent IDs
agent_id_map = {old_id: new_id for new_id, old_id in enumerate(sorted(df['i'].unique()))}
agent_id_mapj = {old_id: new_id for new_id, old_id in enumerate(sorted(df['j'].unique()))}
t_map = {old_id: new_id for new_id, old_id in enumerate(sorted(df['t'].unique()))}
weight_map = {old_id: 1 for new_id, old_id in enumerate(sorted(df['weight']))}

In [ ]:
# Map the old agent IDs to the new agent IDs in the DataFrame
df['i'] = df['i'].map(agent_id_map)
df['j'] = df['j'].map(agent_id_mapj)
df['t'] = df['t'].map(t_map)
df['weight'] = df['weight'].map(weight_map)
df

Build the Network by initializing a DBN


In [ ]:
dbn = DelayBufferNetwork(nettype = "wd", from_df=df, uniform_time_range=True, dont_build_df=True, del_df=True)
# save it in case you need it later
dbn.save_event_arrays("Thiers13/1")
dbn.save_event_dict("Thiers13/1")

First, do a practice run

In [ ]:
def sim(b):
    dbn = DelayBufferNetwork(load=True, path="Thiers13/1")
    dbn.add_delay(expon_distr_bool=True, tau = 1, using_event_dict=True)
    dbn.add_event_buffer(buffer=b, uniform_buffer_bool=True, using_event_dict=True)
    dbn.process_delays_fast_arrays(interact_with_topology=False)
    event_delays = dbn.event_current_delay_array
    agent_delays = np.mean(dbn.agent_delays, axis=1)
    return event_delays, agent_delays

In [ ]:
result = sim(b=1)

Let's see the results of this single run.

In [ ]:
plt.plot(range(len(result[1])), result[1])
plt.xlabel("Time steps")
plt.ylabel("Mean delay per node")

To create a $v$ versus $B$ graph, we must do more simulations.

Let's set our simulation settings

In [ ]:
buffers = np.arange(0, 5, 0.5)

Simulate! (it roughly takes the same time as the single simulation / number of threads * 50)

In [ ]:
from joblib import Parallel, delayed
results = Parallel(n_jobs=4)(delayed(sim)(buffers[i]) for i in range(len(buffers)))

Reformat the results to give us $V$

In [ ]:
delays_prop = []
for i in range(len(results)):
    delays_prop.append(results[i][1][1:] - results[i][1][:-1])

Now we can plot a simple $v$ versus $B$ graph

In [ ]:
for i, b in enumerate(buffers):
    plt.scatter(b, np.mean(delays_prop[i]), c="r")
    
plt.legend(["Workplace15"])

plt.xlabel(r"$B$")
plt.ylabel(r"$v$")

plt.show()